In [44]:
%pip install cplex
%pip install docplex

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.comNote: you may need to restart the kernel to use updated packages.



Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [45]:
from typing import Tuple, List
import logging

def read_hgr(file_path: str) -> Tuple[int, int, List[List[int]]]:
    """
    Read a DIMACS-like .hgr file and return the number of nodes, edges, and the edge list.

    Args:
        file_path: Path to the .hgr file

    Returns:
        Tuple containing:
        - number of nodes
        - number of edges
        - list of edges, where each edge is a list of vertex indices (1-based)

    Example format:
        c I am a comment
        p hs 6 5
        1 2
        2 3 4
        2 4 5
        1 3 6
        5 6
    """
    num_nodes = 0
    num_edges = 0
    edges = []

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()

            # Skip comments and empty lines
            if not line or line.startswith('c'):
                continue

            # Parse problem line
            if line.startswith('p'):
                parts = line.split()
                if len(parts) != 4 or parts[1] != 'hs':
                    raise ValueError(f"Invalid problem line format: {line}")
                num_nodes = int(parts[2])
                num_edges = int(parts[3])
                continue

            # Parse edge line
            vertices = [int(v) for v in line.split()]
            if not vertices:
                continue

            # Validate vertex numbers
            if any(v < 1 or v > num_nodes for v in vertices):
                raise ValueError(f"Vertex number out of range in line: {line}")

            edges.append(vertices)

    if not num_nodes or not num_edges:
        raise ValueError("Problem line not found or invalid")

    if len(edges) != num_edges:
        logging.warning(f"Expected {num_edges} edges but found {len(edges)}")

    return num_nodes, num_edges, edges


import docplex.mp.model as cplex

def solve_ilp(num_nodes, num_edges, edges, param_dict={}, print_info=False):
    mdl = cplex.Model()

    picked_vertices = mdl.binary_var_list(num_nodes, name="picked_vertices")

    for idx, edge in enumerate(edges):
        mdl.add_constraint(mdl.sum(picked_vertices[node-1] for node in edge) >= 1, "Edge_" + str(idx))

    mdl.minimize(mdl.sum(picked_vertices))

    """Set parameters on the model using a dictionary."""
    for param_path, value in param_dict.items():
        parts = param_path.split('.')
        param = mdl.parameters
        for part in parts:
            param = getattr(param, part)
        param.set(value)

    solution = mdl.solve()
    if print_info:
        mdl.print_information()
        print(mdl.parameters._params)
        print(mdl.solve_details)
        print(f"Solution size: {solution.objective_value}")
        # print(f"Sol: {solution}")
    result = {
        "solve_details": mdl.solve_details,
        "config": param_dict,
        "objective": solution.objective_value if solution else None,
        "solve_status": mdl.solve_details.status,
        "time": mdl.solve_details.time,
        "gap": mdl.solve_details.mip_relative_gap,
        "nodes": mdl.solve_details.nb_nodes_processed
    }

    return result

In [ ]:
FOLDER = 200
import os
if not os.path.exists(f"{FOLDER}"):
	os.mkdir(f"{FOLDER}")
HGRPATH = f"./testset/bremen_subgraph_{FOLDER}.hgr"

num_nodes, num_edges, edges = read_hgr(HGRPATH)
print(f"Graph has {num_nodes} nodes and {num_edges} edges")
solve_ilp(num_nodes, num_edges, edges, print_info=True)

Graph has 63 nodes and 63 edges
Model: docplex_model40987
 - number of variables: 63
   - binary=63, integer=0, continuous=0
 - number of constraints: 63
   - linear=63
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
[docplex.mp.params.IntParameter(parameters.advance,1), docplex.mp.params.IntParameter(parameters.clocktype,2), docplex.mp.params.NumParameter(parameters.dettimelimit,1e+75), docplex.mp.params.IntParameter(parameters.lpmethod,0), docplex.mp.params.IntParameter(parameters.optimalitytarget,0), docplex.mp.params.IntParameter(parameters.parallel,0), docplex.mp.params.BoolParameter(parameters.paramdisplay,1), docplex.mp.params.IntParameter(parameters.qpmethod,0), docplex.mp.params.IntParameter(parameters.randomseed,202009243), docplex.mp.params.BoolParameter(parameters.record,0), docplex.mp.params.IntParameter(parameters.solutiontype,0), docplex.mp.params.IntParameter(parameters.threads,0), docplex.mp.params.NumParameter(parameters.timelimit,1e+75), docpl

{'solve_details': docplex.mp.SolveDetails(time=0.078,status='integer optimal solution'),
 'config': {},
 'objective': 17.0,
 'solve_status': 'integer optimal solution',
 'time': 0.07800000000133878,
 'gap': 0.0,
 'nodes': 0}

In [47]:
from tqdm.contrib.itertools import product
# [docplex.mp.params.IntParameter(parameters.advance,1), docplex.mp.params.IntParameter(parameters.clocktype,2), docplex.mp.params.NumParameter(parameters.dettimelimit,1e+75), docplex.mp.params.IntParameter(parameters.lpmethod,0), docplex.mp.params.IntParameter(parameters.optimalitytarget,0), docplex.mp.params.IntParameter(parameters.parallel,0), docplex.mp.params.BoolParameter(parameters.paramdisplay,1), docplex.mp.params.IntParameter(parameters.qpmethod,0), docplex.mp.params.IntParameter(parameters.randomseed,202009243), docplex.mp.params.BoolParameter(parameters.record,0), docplex.mp.params.IntParameter(parameters.solutiontype,0), docplex.mp.params.IntParameter(parameters.threads,0), docplex.mp.params.NumParameter(parameters.timelimit,1e+75), docplex.mp.params.StrParameter(parameters.workdir,.), docplex.mp.params.NumParameter(parameters.workmem,2048.0)]

import pandas as pd
import numpy as np

NR_RUNS = 30
def gridsearch(param_values, runs_per_data = NR_RUNS):
	rezults = []
	param_names = list(param_values.keys())
	for cfg in product(*param_values.values()):
		# Remove None values (-> defaults)
		cfg = {param_names[i]: v for i, v in enumerate(cfg) if v is not None}
		times = [solve_ilp(num_nodes, num_edges, edges, param_dict=cfg)['time'] for _ in range(runs_per_data-1)]
		rez = solve_ilp(num_nodes, num_edges, edges, param_dict=cfg)
		times.append(rez['time'])
		rez['time_mean'] = np.mean(times)
		rez['time_std'] = np.std(times)
		rez['time_var'] = np.var(times)
		del rez['time']
		rezults.append(rez)
	return rezults

In [48]:
# 138:49:35 HOURS cu toate (cu o singura rulare)
param_values_tols_strat = {
	"threads": [1],
	"mip.tolerances.mipgap":[None, 0.01, 0.05, 0.1],
	"mip.tolerances.integrality":[None, 1e-5, 1e-4, 1e-3],
	"mip.strategy.search": [None, 0, 1, 2],#auto trad dynamic
	# "lpmethod": [None, 1, 2, 3], #primal, dual, barrier
	# "mip.cuts.gomory": [None, 0, 1, 2], #off, moderate, aggressive
	# "mip.cuts.liftproj": [None, 0, 1, 2],
	# "mip.cuts.mircut": [None, 0, 1, 2],
	# "mip.cuts.zerohalfcut": [None, 0, 1, 2],
	# "mip.cuts.flowcovers": [None, 0, 1, 2],
}

# 138:49:35 HOURS cu toate (cu o singura rulare)
rezults_tols_strat = gridsearch(param_values_tols_strat)
rezults_tols_strat = pd.DataFrame(rezults_tols_strat).sort_values(['objective', 'time_mean'], ascending=[True, True])
display(rezults_tols_strat)
rezults_tols_strat.to_json(f"{FOLDER}/rezults_tols_stratt1.json")

  0%|          | 0/64 [00:00<?, ?it/s]

,solve_details,config,objective,solve_status,gap,nodes,time_mean,time_std,time_var
41,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.tolerances.mipgap': 0.05, ...",17.0,integer optimal solution,0.000000,0,0.012467,0.006249,0.000039
46,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.tolerances.mipgap': 0.05, ...",17.0,integer optimal solution,0.000000,0,0.013000,0.005831,0.000034
8,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.tolerances.integrality': 0...",17.0,integer optimal solution,0.000000,0,0.013000,0.005831,0.000034
13,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.tolerances.integrality': 0...",17.0,integer optimal solution,0.000000,0,0.013500,0.005315,0.000028
45,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.tolerances.mipgap': 0.05, ...",17.0,integer optimal solution,0.000000,0,0.013533,0.005327,0.000028
...,...,...,...,...,...,...,...,...,...
48,"status = integer optimal, tolerance\ntime ...","{'threads': 1, 'mip.tolerances.mipgap': 0.1}",18.0,"integer optimal, tolerance",0.073326,0,0.006767,0.007745,0.000060
49,"status = integer optimal, tolerance\ntime ...","{'threads': 1, 'mip.tolerances.mipgap': 0.1, '...",18.0,"integer optimal, tolerance",0.073326,0,0.006767,0.007745,0.000060
58,"status = integer optimal, tolerance\ntime ...","{'threads': 1, 'mip.tolerances.mipgap': 0.1, '...",18.0,"integer optimal, tolerance",0.073326,0,0.007300,0.007811,0.000061
55,"status = integer optimal, tolerance\ntime ...","{'threads': 1, 'mip.tolerances.mipgap': 0.1, '...",18.0,"integer optimal, tolerance",0.073326,0,0.007300,0.007811,0.000061


In [49]:
param_values_bnb = {
	"threads": [1],
	"mip.strategy.search": [1],#b&b
	"lpmethod": [1, 2], #primal, dual
	"mip.cuts.gomory": [0, 1, 2], #off, moderate, aggressive
	"mip.cuts.liftproj": [0, 1, 2],
	"mip.cuts.mircut": [0, 1, 2],
	"mip.cuts.zerohalfcut": [0, 1, 2],
}

rezults_bnb = gridsearch(param_values_bnb)
rezults_bnb = pd.DataFrame(rezults_bnb).sort_values(['objective', 'time_mean'], ascending=[True, True])
display(rezults_bnb)
rezults_bnb.to_json(f"{FOLDER}/rezults_bnbt1.json")

  0%|          | 0/162 [00:00<?, ?it/s]

,solve_details,config,objective,solve_status,gap,nodes,time_mean,time_std,time_var
138,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",17.0,integer optimal solution,0.0,0,0.009367,0.007657,0.000059
140,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",17.0,integer optimal solution,0.0,0,0.009900,0.007543,0.000057
141,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",17.0,integer optimal solution,0.0,0,0.009900,0.007543,0.000057
147,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",17.0,integer optimal solution,0.0,0,0.009900,0.007543,0.000057
135,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",17.0,integer optimal solution,0.0,0,0.009933,0.007567,0.000057
...,...,...,...,...,...,...,...,...,...
130,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",17.0,integer optimal solution,0.0,0,0.016600,0.004013,0.000016
19,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",17.0,integer optimal solution,0.0,0,0.016600,0.004144,0.000017
21,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",17.0,integer optimal solution,0.0,0,0.016733,0.003838,0.000015
48,status = integer optimal solution\ntime = ...,"{'threads': 1, 'mip.strategy.search': 1, 'lpme...",17.0,integer optimal solution,0.0,0,0.017267,0.004711,0.000022


In [50]:
param_values_th = {
	"threads": [None, 1],
}

rezults_th = gridsearch(param_values_th)
rezults_th = pd.DataFrame(rezults_th).sort_values(['objective', 'time_mean'], ascending=[True, True])
display(rezults_th)

  0%|          | 0/2 [00:00<?, ?it/s]

,solve_details,config,objective,solve_status,gap,nodes,time_mean,time_std,time_var
1,status = integer optimal solution\ntime = ...,{'threads': 1},17.0,integer optimal solution,0.0,0,0.0135,0.005315,0.000028
0,status = integer optimal solution\ntime = ...,{},17.0,integer optimal solution,0.0,0,0.0313,0.019362,0.000375
